Importe

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

Welche Daten sind verfügbar und Laden des Datensatzes

In [2]:
col_list= ["case_status","case_number","case_no", "decision_date", "case_received_date"]
visas_df = pd.read_csv("us_perm_visas.csv", usecols=col_list)
visas_df.head ()

C:\Users\lassu\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (5,6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,case_no,case_number,case_received_date,case_status,decision_date
0,A-07323-97014,NaN,NaN,Certified,2012-02-01
1,A-07332-99439,NaN,NaN,Denied,2011-12-21
2,A-07333-99643,NaN,NaN,Certified,2011-12-01
3,A-07339-01930,NaN,NaN,Certified,2011-12-01
4,A-07345-03565,NaN,NaN,Certified,2012-01-26


Untersuchung der vorliegenden Daten im Datensatz


In [3]:
# df = pd.read_csv('us_perm_visas.csv', low_memory = False, parse_dates=['decision_date', 'case_received_date'])
visas_df ["decision_date"]=pd.to_datetime(visas_df["decision_date"])
visas_df ["case_received_date"]=pd.to_datetime(visas_df["case_received_date"])

Überprüfung der Anzahl der Visa Bewerbungen bzw. Fälle (Zeilen und Spalten)


In [4]:
print('Number of Visa Applications:', len(visas_df))
print('Number of Columns:', len(visas_df.columns))

Number of Visa Applications: 374362
Number of Columns: 5


Ersten fünf Zeilen anzeigen lassen zum checken

In [5]:
visas_df.head()

,case_no,case_number,case_received_date,case_status,decision_date
0,A-07323-97014,NaN,NaT,Certified,2012-02-01
1,A-07332-99439,NaN,NaT,Denied,2011-12-21
2,A-07333-99643,NaN,NaT,Certified,2011-12-01
3,A-07339-01930,NaN,NaT,Certified,2011-12-01
4,A-07345-03565,NaN,NaT,Certified,2012-01-26


Letze fünf Zeilen anzeigen lassen zum checken

In [6]:
visas_df.tail()

,case_no,case_number,case_received_date,case_status,decision_date
374357,NaN,A-16363-85407,2016-12-29,Withdrawn,2016-12-30
374358,NaN,A-16271-56745,2016-12-30,Withdrawn,2016-12-30
374359,NaN,A-16354-82345,2016-12-30,Withdrawn,2016-12-30
374360,NaN,A-16357-84250,2016-12-30,Withdrawn,2016-12-30
374361,NaN,A-16279-59292,2016-12-30,Withdrawn,2016-12-30


Namen der Spalten anzeigen lassen



In [7]:
print(visas_df.columns.values)

['case_no' 'case_number' 'case_received_date' 'case_status'
 'decision_date']


Die Werte case_number und case_no könnten ähnliche Informationen beinhalten. Daher überprüfen wir  Länge, Anzahl der fehlenden Werte und Beispielwerte

In [8]:
print("Length of 'case_number' column is: ", len(visas_df['case_number'])," with",visas_df.case_number.isnull().sum(), "missing values")
print("Length of 'case_no' column is: ", len(visas_df['case_no'])," with", visas_df.case_no.isnull().sum(),"missing values \n")

print("First 2 values of case_number column are : \n", visas_df['case_number'].head(2),"\n")

print("Last 2 values of case_number column are : \n", visas_df['case_number'].tail(2), "\n")
print("First 2 values of case_no column are : \n", visas_df['case_no'].head(2), "\n")
print("Last 2 values of case_no column are : \n", visas_df['case_no'].tail(2))

Length of 'case_number' column is:  374362  with 135269 missing values
Length of 'case_no' column is:  374362  with 239093 missing values 

First 2 values of case_number column are : 
 0    NaN
1    NaN
Name: case_number, dtype: object 

Last 2 values of case_number column are : 
 374360    A-16357-84250
374361    A-16279-59292
Name: case_number, dtype: object 

First 2 values of case_no column are : 
 0    A-07323-97014
1    A-07332-99439
Name: case_no, dtype: object 

Last 2 values of case_no column are : 
 374360    NaN
374361    NaN
Name: case_no, dtype: object


Wie erwartet sind die Werte in case_number und case_no ähnlich. Eine Addition der NaN Werte der Spalten case_number und case_no ergeben die Gesamtzahl der Fälle.
Daher erstellen wir eine neue Spalte die keine fehlenden Werte beinhaltet.

In [9]:
case_noindex = visas_df.columns.loc('case_no')
case_noindexnumber = visas_df.columns.loc('case_number')
casenumberlist = []


for value in visas_df.iloc[0:135269,case_noindex]:
    casenumberlist.append(value)

for value in visas_df.iloc[135269:374362,case_noindexnumber]:
    casenumberlist.append(value)

visas_df['casenumber'] = casenumberlist
visas_df.drop(visas_df.columns[[case_noindex,case_noindexnumber]], axis=1, inplace=True)

AttributeError: 'Index' object has no attribute 'loc'

Überprüfung der Spalte case_status, da sie Informationen über den jeweiligen Visumsangetrag gentroffene Entscheidung enthaelt

In [ ]:
for value in visas_df.case_status.unique():
    print(len(visas_df[visas_df['case_status'] == value])," occurrences of status '{}'".format(value))

Datensatz enthält 18194 Fälle die zurückgezogen wurden 'withdrawn'. Da diese Fälle nicht relevant sind werden sie aus dem Datensatz gelöscht.

In [ ]:
visas_df= visas_df[visas_df.case_status != 'Withdrawn']


Der Status 'certified' und der Status 'certified-expired' werden zu dem Wert 'certified' zusammengefasst, sodass es nur noch die Werte 'certified' und 'denied' hat.

Antragssteller des Visas haben nach Erhalt des Status 'certified' 6 Monate Zeit, das Formular I-140 einzureichen, bevor es abläuft und in den Status 'certified-expired' erhält.

Formular I-140 Quelle: Wikipedia
Das Formular I-140, Immigrant Petition for Alien Worker (Einwanderungsantrag für ausländische Arbeitnehmer), ist ein Formular, das von einem potentiellen Arbeitgeber bei den United States Citizenship and Immigration Services (USCIS) eingereicht wird, um einen Ausländer zu ersuchen, dauerhaft in den USA zu arbeiten. Dies geschieht in dem Fall, dass der Arbeitnehmer in gewisser Weise als außergewöhnlich gilt oder wenn es in den USA keine qualifizierten Arbeitskräfte gibt.


In [ ]:
visas_df.loc[visas_df.case_status == 'Certified-Expired', 'case_status'] = 'Certified'
visas_df.case_status.value_counts()

